# 型変換と演算子オーバーロード


## キーポイント

*


----

## 1 型変換

----


### 1.1 文字列クラスのインターフェイスを考える

`int`や`double`のような組み込み型には、さまざまな演算が定義されています。<br>
また、論理的に近い型同士では、相互に変換が可能です。<br>
例えば、`int`と`double`の演算では、コンパイラは自動的に`int`を`double`に変換します。

クラスの設計者は、クラスの生成、コピー、代入、破棄の内容を自由に決められます。<br>
自動的な変換を可能にするには、コピーと代入を適切に定義します。

本テキストでは、クラスの型変換と、それに関連する関数を作成し、組み込み型のように扱えるクラスを作成します。<br>
このクラスの名前は`EasyStr`(イージー・エスティーアール)`とします。

`EasyStr`のお手本として、標準ライブラリの`string`クラスを参考にします。<br>
`string`クラスは、さまざまなパターンの型変換と関連する関数を持っているからです。<br>
とはいえ、`string`クラスのインターフェイスは`vector`より多いので、作成するのは機能を絞ったバージョンになります。

それでは、必要なインターフェイスを決めるために、`string`クラスの使い方を振り返ってみましょう。

```cpp
// string型の変数宣言
string s;
string t = "abcdef";
string u = t;

// サイズと添字を使ったループ
for (int i = 0; i < (int)t.size(); i++) {
  cin >> t[i];
}

// イテレータを使う
sort(t.begin(), t.end());

// 要素の追加と削除
s.push_back('a');
s.pop_back();

// 文字列の代入
s = t;
s = "abc";

// 標準入出力の読み書き
cin >> s;
cout << s;

// 文字列の加算
s += t;
t = s + t;
s += "def";
t = s + "ghi";
t = "jkl" + s;
```

上記のプログラムでは、`string`の以下の機能を使用しています。

1. パラメータなしの宣言
2. 文字列を指定して初期化
3. サイズの取得
4. 添字によるデータの読み書き
5. イテレータの取得
6. 要素の追加と削除
7. `string`型の代入
8. 文字列の代入
9. 標準入力からの読み込み
10. 標準出力への書き込み
11. `string`の加算代入
12. `string`の加算
13. 文字列の加算代入

`string`は`vector`と同様の機能に加えて、文字列に関する機能を持つので、そのぶんインターフェイスが多くなります。


#### ファイルの準備

本テキストでは、実際に`EasyStr`クラスを作成してもらいます。<br>
`EasyStr`クラスはクラステンプレート**ではない**ので、ヘッダファイルにインターフェイスを書き、ソースファイルにメンバ関数の定義を書きます。
そのために、`EasyStr.h`と`EasyStr.cpp`という2つのファイルを用意します。

`Google Colab`では、コードセルの先頭に`%%writefile ファイル名`というマジックコマンドを書くことでファイルを作成できます。

この下に、マジックコマンドを書いた`EasyStr.h`と`EasyStr.cpp`という、2つのコードセルをご用意しました。

タイトルの`EasyStr.h`、`EasyStr.cpp`という文字をクリックすると、右側にアイコンリストが表示されます。アイコンリストから「タブのミラーセル」という説明が出るアイコンをクリックして、セルを複製してください(他のアイコンをクリックしないように注意)。


In [ ]:
# @title EasyStr.h
# タブの「ミラーセル」機能で、このセルを複製してください。
%%writefile EasyStr.h


In [ ]:
# @title EasyStr.cpp
# タブの「ミラーセル」機能で、このセルを複製してください。
%%writefile EasyStr.cpp


ウィンドウの右側に、`EasyStr.h`と`EasyStr.cpp`という2つのタブが表示されていたらOKです。以後、「`EasyStr.h`(または`EasyStr.cpp`)に次のプログラムを追加(あるいは削除、変更)してください」というテキストを見たら、右に表示されている`EasyStr.h`、`EasyStr.cpp`タブのコードを変更してください。

変更後に`▶`ボタンをクリックして、ファイルを保存すること。

また、`EasyStr`クラスでは、動的メモリ管理を`EasyVec`クラスに任せるつもりです。<br>
残念ながら、Google Colabには、別のノートのセルを参照する仕組みがありません。そこで、手動でコピーを作成します。<br>
前回の「C++言語\_第17回\_クラスの設計と実装」から、自分で作成した`EasyVec.h`の内容をコピーし、以下のセルに貼り付けてください。


In [ ]:
# @title EasyVec.h
# タブの「ミラーセル」機能で、このセルを複製してください。
%%writefile EasyVec.h
// この下に、前回テキストからコピーしたEasyVec.hを貼り付けてください。


`EasyVec.h`の完成版を貼り付けたら、`▶`ボタンをクリックしてファイルを保存してください。<br>
保存したら「タブのミラーセル」アイコンをクリックして、セルの複製を作成してください。

今回、`EasyVec.h`には手を付けませんが、サーバとの接続が切れた場合は、`▶`ボタンをクリックして保存し直す必要があります。


### 1.3 EasyStrクラスの定義

インターフェイスが決まったので、次は`EasyStr`クラスを具体的に定義していきます。<br>
`string`はテンプレートを使っていないので、`EasyStr`クラスも普通のクラスとして定義します。

また、メモリの管理に`EasyVec`クラスを使うので、`EasyVec.h`をインクルードします。

`EasyStr.h`の`%%writefile`行の下に、次のプログラムを追加してください。追加したら`▶`ボタンを押して保存すること。

```cpp
#ifndef EASYSTR_H
#define EASYSTR_H
#include "EasyVec.h"

class EasyStr {
public:

private:
  EasyVec<char> data;
};

#endif // EASYSTR_H
```

>追加のしかたが分からない場合は、以下の「追加例」の「コードの表示」をクリックして、参考にしてください


In [ ]:
# @title 🧩追加例
%%writefile EasyStr.h
#ifndef EASYSTR_H
#define EASYSTR_H
#include "EasyVec.h"

class EasyStr {
public:

private:
  EasyVec<char>  data;
};

#endif // EASYSTR_H

### 1.4 型の定義

`EasyVec`クラスと同様に、`EasyStr`クラスにも以下の型の定義があると共通のプログラムを書きやすくなります。

* iterator:&emsp;&emsp;&emsp;基本となるイテレータ型
* const_iterator: 指している値を書き換えられないイテレータ型
* size_type:&emsp;&emsp; 配列の大きさなどを表現する型
* value_type:&emsp;&ensp;コンテナに格納するデータ型

今回は、メモリ管理用に`EasyVec`クラスを使っているので、`EasyVec`クラスの型の定義を利用できます。

例えば、`iteretor`型の定義は、次のように書けます。

`using iterator = EasyVec<char>::iterator;`

`EasyVec<char>`と`iterator`の間にある2つのコロン記号(`::`)は、「スコープ解決演算子」といいます。

実は、構造体やクラスの定義も「スコープ」になります。<br>
すべてのメンバは、構造体やクラスのスコープに含まれます。

`クラス名::メンバ名`のように書くことで、クラスのメンバを指定できます。

それでは、必要な型を追加しましょう。`EasyStr.h`に、次のプログラムを追加してください。

**diff**

<pre><code><div> #include "EasyVec.h"

 class EasyStr {
 public:
<font color="#3a3">+  // 型の定義
+  using iterator = EasyVec<char>::iterator;
+  using const_iterator = EasyVec<char>::const_iterator;
+  using size_type =  EasyVec<char>::size_type;
+  using value_type =  EasyVec<char>::value_type;
</font>
 private:
   EasyVec&lt;char&gt; data;
 };
</div></code></pre>


### 1.5 コンストラクタとデストラクタ

`EasyStr`型には、2つのコンストラクタ、コピーコンストラクタ、そしてデストラクタを定義します。<br>
ただし、ヘッダファイルには追加するのは宣言だけです。<br>
`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div> class EasyStr {
 public:
   // 型の定義
   using iterator = EasyVec<char>::iterator;
   using const_iterator = EasyVec<char>::const_iterator;
   using size_type =  EasyVec<char>::size_type;
   using value_type =  EasyVec<char>::value_type;
<font color="#3a3">+
+  // コンストラクタ
+  EasyStr();
+  EasyStr(const char* s);
+
+  // コピーコンストラクタ
+  EasyStr(const EasyStr& other);
+
+  // デストラクタ
+  ~EasyStr() = default;
</font>
 private:
   EasyVec&lt;char&gt; data;
 };
</div></code></pre>

デフォルトコンストラクタは、`\0`だけを持つ文字列を作成します。<br>
文字列を受け取るコンストラクタは、`const char*`型を使って文字列を受け取ります。<br>

デストラクタは、`default`指定をしてコンパイラに任せます。動的メモリ管理は`EasyVec`クラスが全てやってくれるからです。

次に、コンストラクタの定義を作成します。`EasyStr.cpp`に次のプログラムを追加してください。

**diff**

<pre><code><div><font color="#3a3">+#include "EasyVec.h";
+#include &lt;string.h&gt;
+using namespace std;
+
+// デフォルトコンストラクタ
+EasyStr::EasyStr() : data(1) {
+}
+
+// 文字列を受け取るコンストラクタ
+EasyStr::EasyStr(const char* s) : data(strlen(s) + 1) {
+  for (int i = 0; i < (int)data.size(); i++) {
+    data[i] = s[i];
+  }
+}
+
+// コピーコンストラクタ
+EasyStr::EasyStr(const EasyStr& other) : data(other.data) {
+}</font>
</div></code></pre>

メンバ関数を定義する場合は、関数名の部分を`クラス名::メンバ関数名`のように書きます。`::`はスコープ解決演算子です。<br>
上記の関数定義の場合は「`EasyStr`クラスの`EasyStr(const char* s)`コンストラクタ」という意味になります。

デフォルトコンストラクタでは、`EasyVec`クラスのコンストラクタの引数に`1`を指定して、長さ`1`の配列を作成します。<br>
`EasyVec`コンストラクタは配列の要素を`0`で初期化します。その結果、`data`は「長さ`0`の文字列」となります。

文字列を受け取るコンストラクタでは、受け取った文字列を`EasyVec`型の`data`メンバ変数にコピーします。<br>
そのため、`data`の初期値には、文字列をコピーできるサイズが必要となります。

必要なサイズは「文字列の長さ + 終端の`\\0`文字」となります。<br>
文字列の長さを求めるには、標準ライブラリの`strlen`関数を使うと簡単です。

`strlen`関数の宣言は、`string.h`に書いてあるので、`string.h`をインクルードしています。<br>
配列のメモリを確保したら、for文を使って1文字ずつデータをコピーします。

コピーコンストラクタは、`EasyVec`クラスのコピーコンストラクタを使って、データをコピーします。


### 1.6 サイズと添え字演算子

`size`メンバ関数は、`EasyStr`が保持する文字列の長さを返します。<br>
`data`メンバ変数のサイズには終端の`nullptr`が含まれますが、文字列の長さは`nullptr`を含まない点に注意が必要です。

添え字演算子は、`EasyVec`クラスの添え字演算子を呼び出すだけです。<br>
`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>
   // コンストラクタ
   EasyStr() = default;
   EasyStr(const char* s);

   // デストラクタ
   ~EasyStr() = default;
<font color="#3a3">+
+  // 文字列の長さ
+  size_type size() const { return data.size() - 1; }
+
+  // 添え字
+  value_type& operator[](size_type i) { return data[i]; }
+  const value_type& operator[](size_type i) const { return data[i]; }
</font>
 private:
   EasyVec&lt;char&gt; data;
};
</div></code></pre>



### 1.7 イテレータ

次に、イテレータを返す`begin`、`end`メンバ関数を定義しましょう。<br>
`EasyStr`の場合、これらのメンバ関数は、`EasyVec`クラスの同名のメンバ関数を呼び出すだけです。

`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>
   // 文字列の長さ
   size_type size() const { return data.size() - 1; }

   // 添え字
   value_type& operator[](size_type i) { return data[i]; }
   const value_type& operator[](size_type i) const { return data[i]; }
<font color="#3a3">+
+  // イテレータ
+  iterator begin() { return data.begin(); }
+  const_iterator begin() const { return data.begin(); }
+  iterator end() { return data.end(); }
+  const_iterator end() const { return data.end(); }
</font>
 private:
   EasyVec&lt;char&gt; data;
};
</div></code></pre>



### 1.8 要素の追加と削除

末尾に要素を追加する`push_back`メンバ関数と、末尾の要素を削除する`pop_back`メンバ関数、それから全て消去する`clear`メンバ関数の3種類は、イテレータと同様に、それぞれ`EasyVec`クラスの同名の関数を呼び出すだけで完成します。

`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   // イテレータ
   iterator begin() { return data.begin(); }
   const_iterator begin() const { return data.begin(); }
   iterator end() { return data.end(); }
   const_iterator end() const { return data.end(); }
<font color="#3a3">+
+  // 末尾に要素を追加
+  void push_back(char c) { data.push_back(c); }
+
+  // 末尾の要素を削除
+  void pop_back() { data.pop.back(); }
+
+  // すべての要素を削除
+  void clear() { data.clear(); }
</font>
 private:
   EasyVec&lt;char&gt; data;
};
</div></code></pre>



### 1.9 代入演算子

`EasyStr`クラスのインターフェイスには、2パターンの代入があります。<br>
ひとつは`EasyStr`を代入し、もうひとつは文字列を代入します。

ですが、実際には`EasyStr`を代入するコピー代入演算子を定義するだけで十分で、文字列用の代入演算子は不要です。<br>
なぜなら文字列は、文字列を受け取る`EasyStr`コンストラクタによって、自動的に`EasyStr`型に変換されるからです。

これは、`int a = 1.5;`と書いたとき、`double`型である`1.5`が自動的に`int`型の`1`に変換されるのと同じです。<br>
このように自動的に型が変換される機能を「暗黙の型変換(あんもくのかたへんかん)」といいます。

`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   // コンストラクタ
   EasyStr();
   EasyStr(const char* s);

   // コピーコンストラクタ
   EasyStr(const EasyStr& other);

   // デストラクタ
   ~EasyStr() = default;
<font color="#3a3">+
+  // コピー代入演算子
+  EasyStr& operator=(const EasyStr& other);
</font>
   // 文字列の長さ
   size_type size() const { return data.size() - 1; }

   // 添え字
   value_type& operator[](size_type i) { return data[i]; }
   const value_type& operator[](size_type i) const { return data[i]; }
</div></code></pre>

続いて、コピー代入演算子を定義します。コピー代入演算子は、`EasyVec`クラスのコピー代入演算子を使ってデータをコピーします。<br>
`EasyStr.cpp`に次のプログラムを追加してください。

**diff**

<pre><code><div> // 文字列を受け取るコンストラクタ
 EasyStr::EasyStr(const char* s) : data(strlen(s) + 1) {
   for (int i = 0; i < (int)data.size(); i++) {
     data[i] = s[i];
   }
 }

 // コピーコンストラクタ
 EasyStr::EasyStr(const EasyStr& other) : data(other.data) {
 }
<font color="#3a3">+
+// コピー代入演算子
+EasyStr& EasyStr::operator=(const EasyStr& other) {
+  data = other.data;
+  return *this;
+}</font>
</div></code></pre>







### 1.10 標準入出力の読み書き

標準入力である`cin`は`istream`(アイ・ストリーム)クラスの変数で、標準入力を受け付けるように特別に初期化されています。<br>
同様に、標準出力`cout`は`ostream`(オー・ストリーム)クラスの変数で、標準出力を受け付けるように特別に初期化されています。<br>

このため、標準入出力に対してデータを読み書きする関数を書く場合は、`istream`または`ostream`クラスを引数にします。

入力文字列を`string`クラスの変数で受け取る場合、

```cpp
cin >> s; // s は string クラスの変数
```

のように書くのでした。ここで登場する`>>`記号も演算子です。演算子は書きかたが特殊な関数に過ぎないので、オーバーロードできます。<br>
そこで、`EasyStr`用に`<<`演算子を関数オーバーロードすれば、`EasyStr`に対して標準入出力からデータを読み書きできるようになります。

ただし、この場合の`>>`演算子はメンバ関数にはできません。メンバ関数にするには、式の左辺のクラスのメンバにする必要があるからです。<br>
`cin >> s;`という式の場合、左辺は`cin`なので、`istream`クラスです。

ところが、`istream`クラスは標準ライブラリに含まれるクラスなので、勝手にメンバ関数を追加することはできません。<br>
このような場合は、メンバ関数ではなく、普通の関数として定義します。

まず、`istream`クラスを使えるように、`iostream`ヘッダファイルをインクルードしましょう。<br>
`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div> #include "EasyVec.h";
<font color="#3a3">+#include &lt;iostream&gt;</font>

 class EasyStr {
 public:
   // 型の定義
   using iterator = EasyVec<char>::iterator;
   using const_iterator = EasyVec<char>::const_iterator;
   using size_type =  EasyVec<char>::size_type;
   using value_type =  EasyVec<char>::value_type;
</div></code></pre>

次に、関数の引数と戻り型を決めます。`cin >> s`の例で考えると、左辺の`cin`が第1引数、右辺の`s`が第2引数に相当します。<br>
`cin`は`istream`クラス、`s`は`EasyStr`クラスです。どちらも変更されうるため、`const`は付けられません。<br>
これらを考え合わせると、引数の型は`istream&`と`EasyStr&`となります。

戻り型については、`cin >> s >> t;`のような記述を可能にしたいです。<br>
そして、`cin >> s >> t;`は`cin >> s; cin >> t;`と同じ挙動になってほしいです。

そのためには、`cin >> s`の戻り値は、`cin`と同等の型でなくてはなりません。<br>
この条件を満たす戻り値の型は、`istream&`になります。

それでは、`>>`演算子の宣言を追加しましょう。<br>
追加する位置は、`istream`だけでなく`EasyStr`クラスの定義も参照できなくてはなりません。<br>
そのため、定義位置はクラス定義より下に書く必要があります。

`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   // 末尾に要素を追加
   void push_back(char c) { data.push_back(c); }

   // 末尾の要素を削除
   void pop_back() { data.pop.back(); }

 private:
   EasyVec&lt;char&gt; data;
};
<font color="#3a3">+
+// ストリームから文字列の読み込む
+istream& operator>>(istream& is, EasyStr& str);</font>

 #endif // EASYSTR_H
</div></code></pre>

続いて`<<`演算子の定義を書きます。`string`と同じように機能させるには、空白を判定して読み込みを停止しなくてはなりません。<br>
空白を判定するには`ctype.h`で定義されている`isspace`関数を使います。

まずは`ctype.h`をインクルードしましょう。`EasyStr.cpp`に次のプログラムを追加してください。

**diff**

<pre><code><div> #include "EasyVec.h";
 #include &lt;string.h&gt;
<font color="#3a3">+#include &lt;ctype.h&gt;</font>
 using namespace std;

 // デフォルトコンストラクタ
 EasyStr::EasyStr() : data(1) {
 }

 // 文字列を受け取るコンストラクタ
 EasyStr::EasyStr(const char* s) : data(strlen(s) + 1) {
   for (int i = 0; i < (int)data.size(); i++) {
     data[i] = s[i];
   }
 }
</div></code></pre>

それでは、`>>`演算子を定義しましょう。`>>`演算子を`string`と同じように機能させるには、以下の処理が必要です。

1. 読み込み先の`EasyStr`を空(から)にする
2. 文字列の前にある空白を読み飛ばす
3. データ終端、または空白が来るまで文字列を読み込む

3の段階で`istream`クラスから文字列を読み込むために、以下の5つの機能を使います。

>```cpp
>istream& ws(istream& is)
>```
>
>`ws`(ダブリューエス、ホワイト・スペースの頭文字)関数は、空白文字を読み捨てるストリーム操作クラス(マニピュレータ)。<br>
>`cin >> ws`のように書くことで、入力から空白文字を読み捨てることができる。

>```cpp
>int istream::get()
>```
>
>`istream::get`(アイストリーム・ゲット)関数は、ストリームから1文字読み込み、読み込んだ文字を返す。<br>
>終端文字列の場合は`char_traits<char>::eof()`を返す。

>```cpp
>tempalte<typename T>
>int char_traits<T>::eof()
>```
>
>`char_traits::eof`(チャートレイツ・イーオーエフ)関数は、終端文字を返す。

>```cpp
>int isspace(int c);
>```
>
>`isspace`(イズ・スペース)関数は、引数で渡された文字が「空白文字」なら`0`以外、空白文字でなければ`0`を返します。

>```cpp
>void istream::unget()
>```
>
>`istream::unget`(アイストリーム・アンゲット)関数は、最後に読み込んだ1文字をストリームに戻して、読まなかったことにします。

`EasyStr.cpp`に、`>>`演算子の定義を追加してください。

**diff**

<pre><code><div>  // コピーコンストラクタ
 EasyStr::EasyStr(const EasyStr& other) : data(other.data) {
 }

 // コピー代入演算子
 EasyStr& EasyStr::operator=(const EasyStr& other) {
   data = other.data;
   return *this;
 }
<font color="#3a3">+
+// ストリームから文字列を読み込む
+istream& operator>>(istream& is, EasyStr& str) {
+  // 読み込み先の`EasyStr`を空にする  
+  str.clear();
+
+  // 文字列の前にある空白を読み飛ばす
+  is >> ws;
+
+  // データ終端、または空白が来るまで文字列を読み込む
+  while (is) {
+    int c = is.get();
+    if (c == char_traits&lt;char&gt;::eof()) {
+      break; // データ終端に到達したので読み込み終了
+    }
+    else if (isspace(c)) {
+      is.unget(); // ストリームに文字を戻す
+      break; // 空白を見つけたので読み込み終了
+    }
+    str.push_back((char)c); // 文字を追加
+  }
+
+  return is;
+}
</font>
</div></code></pre>








### 1.11 文字列の連結

お手本となる`string`クラスの変数は、`+=`演算子や`+`演算子を使って「文字列を連結」することができます。<br>
そこで、`EasyStr`クラスも、これらの演算子で連結できるようにしましょう。

引数や戻り型を考える前に、これらの演算子をメンバ関数にするか、通常の関数にするかを決める必要があります。

`a += b`という式を考えてみましょう。この式では、`+=`の左辺にある変数が変更され、右辺は変更されません。<br>
このことから、`+=`演算子は、変更される左辺のクラスのメンバ関数にできそうです。<br>
右辺は変更されないため、引数の型は`const EasyStr&`とするのが適当でしょう。

`string`型の`+=`演算子の場合、戻り型は`string&`で、左辺のオブジェクトが返されます。<br>
極力お手本に合わせて作りたいので、戻り型は`EasyStr&`としましょう。

次に`a + b`という式を考えてみます。この式の結果は`a`と`b`を連結した新しいオブジェクトになります。`a`と`b`の内容は変更されません。<br>
両辺ともに変更されないので、メンバ関数にする必然がありません。ということは、普通の関数にするべきでしょう。

両辺共に変更されないので、引数の型はどちらも`const EasyStr&`となります。<br>
また，式の結果として新しいオブジェクトを返す必要があるため、戻り型は参照なしの`EasyStr`しか選びようがありません。

上記の方針に従って、文字列を連結する`+=`演算子と`+`演算子を追加しましょう。`EasyStr.h`に次のプログラムを追加してください。

**diff**

<pre><code><div>   // 末尾に要素を追加
   void push_back(char c) { data.push_back(c); }

   // 末尾の要素を削除
   void pop_back() { data.pop.back(); }
<font color="#3a3">+
+  // 文字列の連結
+  EasyStr& operator+=(const EasyStr& other);
</font>

 private:
   EasyVec&lt;char&gt; data;
};

 // ストリームから文字列の読み込む
 istream& operator>>(istream& is, EasyStr& str);
<font color="#3a3">+
+// 文字列の連結
+EasyStr operator+(const EasyStr& a, const EasyStr& b);</font>

 #endif // EASYSTR_H
</div></code></pre>




In [ ]:
# @title 🔒️EasyStr.hの最終版(どうしてもバグが取れない場合に見てください)
%%writefile EasyStr.h


In [ ]:
# @title 🔒️EasyStr.cppの最終版(どうしてもバグが取れない場合に見てください)
%%writefile EasyStr.cpp


----

## 2 練習問題

----

以下の手順にしたがって、各問題のプログラムを完成させなさい。

1. `%%writefile ...`の下の行からがプログラムです。問題文に従ってプログラムを修正、または追加してください。
2. プログラムを修正したら、セルの右側にある`▶`をクリックします。すると、ファイルが保存されます。
3. 「動作テスト」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、実行結果が表示されます。<br>
   このセルは、修正したプログラムの動作を確認するために使ってください。
4. 「実行」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、結果の成否が判定されます。
5. 判定に成功したら`AC`と表示されます。次の問題に進んでください。
6. 失敗したら`WA`と表示されます(その前にエラーメッセージが表示される場合もあります)。<br>
   これは、プログラムのどこかにエラーがあることを意味します。<br>
   「動作テスト」を使ってエラーを修正し、`AC`を目指してください。


### ❓️問題１

本テキストで作成した`EasyVec.h`を使って、配列にデータを読み込み、逆順で出力するプログラムを作成しなさい。<br>
最後に改行を出力すること。

>問題を解く前に、`EasyVec.h`セルを実行してファイルを保存すること。

**入力データ形式**

```txt
要素数N
整数A1 整数A2 ... 整数An
```

**入力例**

```txt
5
1 2 3 4 5
```

**出力例**

```txt
5 4 3 2 1
```


In [ ]:
%%writefile practice_01a.cpp
#include "EasyVec.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、要素数と数値を読み込んで逆順で出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "この下をクリックして、要素数と数値の配列を入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "5 4 3 2 1\n7") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "5 1 2 3 4 5" | ./practice_01a && echo "1 7" | ./practice_01a) > nil && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include "EasyVec.h"
#include <iostream>
using namespace std;

int main() {
  // この下に、要素数と数値を読み込んで逆順で出力するプログラムを書く

  int n;
  cin >> n;

  EasyVec<int> v(n);
  for (int i = 0; i < n; i++) {
    cin >> v[i];
  }

  for (int i = n - 1; i >= 0; i--) {
    cout << v[i] << " ";
  }
  cout << endl;
}